# Running Bismark

In [1]:
wd="/Volumes/Orange_block/wd/18-06-13"

In [ ]:
mkdir $wd

In [2]:
cd $wd

/Volumes/Orange_block/wd/18-06-13


## Genome Preparation

In [3]:
!mkdir genome

In [4]:
!cp /Users/sr320/Desktop/oly/Olurida_v081.fa genome//Olurida_v081.fa

In [5]:
%%bash
/Applications/bioinfo/Bismark_v0.19.0/bismark_genome_preparation \
--path_to_bowtie /Applications/bioinfo/bowtie2-2.3.4.1-macos-x86_64 \
genome


Total number of conversions performed:
C->T:	197058296
G->A:	197049137
Please be aware that this process can - depending on genome size - take several hours!
Settings:
  Output files: "BS_CT.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  genome_mfa.CT_conversion.fa
Reading reference sizes
Settings:
  Output files: "BS_GA.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, 

Writing bisulfite genomes out into a single MFA (multi FastA) file

Bisulfite Genome Indexer version v0.19.0 (last modified 07 November 2016)

Step I - Prepare genome folders - completed



Step II - Genome bisulfite conversions - completed


Bismark Genome Preparation - Step III: Launching the Bowtie 2 indexer
Building a SMALL index
Building a SMALL index


In [7]:
!ls genome/Bisulfite_Genome/

CT_conversion GA_conversion


# Download Seq Read Files

In [14]:
!mkdir data

In [23]:
!wget --accept "[1-8]_*.gz" --reject "index" --recursive -nd -np \
http://owl.fish.washington.edu/nightingales/O_lurida/


--2018-06-13 12:33:22--  http://owl.fish.washington.edu/nightingales/O_lurida/
Resolving owl.fish.washington.edu (owl.fish.washington.edu)... 128.95.149.83
Connecting to owl.fish.washington.edu (owl.fish.washington.edu)|128.95.149.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.tmp’

index.html.tmp          [ <=>                ]  31.90K  --.-KB/s    in 0.002s  

2018-06-13 12:33:23 (15.5 MB/s) - ‘index.html.tmp’ saved [32669]

Loading robots.txt; please ignore errors.
--2018-06-13 12:33:23--  http://owl.fish.washington.edu/robots.txt
Reusing existing connection to owl.fish.washington.edu:80.
HTTP request sent, awaiting response... 404 Not Found
2018-06-13 12:33:23 ERROR 404: Not Found.

Removing index.html.tmp since it should be rejected.

--2018-06-13 12:33:23--  http://owl.fish.washington.edu/nightingales/O_lurida/?C=N;O=D
Reusing existing connection to owl.fish.washington.edu:80.
HTTP request sent, awaiting 

In [25]:
ls

data/   genome/


In [24]:
!mv *gz data/

In [26]:
ls data/

1_ATCACG_L001_R1_001.fastq.gz  5_ACAGTG_L001_R1_001.fastq.gz
2_CGATGT_L001_R1_001.fastq.gz  6_GCCAAT_L001_R1_001.fastq.gz
3_TTAGGC_L001_R1_001.fastq.gz  7_CAGATC_L001_R1_001.fastq.gz
4_TGACCA_L001_R1_001.fastq.gz  8_ACTTGA_L001_R1_001.fastq.gz


# Bismark Mapping

In [27]:
%%bash
find data/*R1* \
| xargs basename -s _L001_R1_001.fastq.gz | xargs -I{} /Applications/bioinfo/Bismark_v0.19.0/bismark \
--path_to_bowtie /Applications/bioinfo/bowtie2-2.3.4.1-macos-x86_64 \
--genome genome \
--score_min L,0,-0.9 \
-u 1000 \
-p 4 \
--non_directional \
data/{}_L001_R1_001.fastq.gz 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Bowtie seems to be working fine (tested command '/Applications/bioinfo/bowtie2-2.3.4.1-macos-x86_64/bowtie2 --version' [2.3.4])
Output format is BAM (default)
Alignments will be written out in BAM format. Samtools found here: '/Applications/bioinfo/samtools-1.7/samtools'
Reference genome folder provided is genome/	(absolute path is '/Volumes/Orange_block/wd/18-06-13/genome/)'
Processing sequences up to read no. 1000 from the input file

Input files to be analysed (in current folder '/Volumes/Orange_block/wd/18-06-13'):
data/1_ATCACG_L001_R1_001.fastq.gz
Library was specified to be not strand-specific (non-directional), therefore alignments to all four possible bis

In [28]:
%%bash

/Applications/bioinfo/Bismark_v0.19.0/deduplicate_bismark \
--bam -s \
*.bam \
2> dedup.err


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
%%bash
/Applications/bioinfo/Bismark_v0.19.0/bismark_methylation_extractor \
--bedGraph --counts --scaffolds \
--multicore 2 \
*deduplicated.bam \
2> bme.err

1_ATCACG_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt.1
1_ATCACG_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt.2

1_ATCACG_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt.1.mbias
1_ATCACG_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt.2.mbias

/Volumes/Orange_block/wd/18-06-13/CpG_OT_1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.txt	/Volumes/Orange_block/wd/18-06-13/CpG_CTOT_1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.txt	/Volumes/Orange_block/wd/18-06-13/CpG_CTOB_1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.txt	/Volumes/Orange_block/wd/18-06-13/CpG_OB_1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.txt

2_CGATGT_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt.1
2_CGATGT_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt.2

2_CGATGT_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt.1.mbias
2_CGATGT_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt.2.mbias

/Volumes/Orange_block/wd/18-06-13/CpG_OT_2_CGATGT_L

In [30]:
!gunzip *bedGraph.gz

In [31]:
%%bash
/Applications/bioinfo/Bismark_v0.19.0/bismark2report

Found 8 alignment reports in current directory. Now trying to figure out whether there are corresponding optional reports

Writing Bismark HTML report to >> 1_ATCACG_L001_R1_001_bismark_bt2_SE_report.html <<

Using the following alignment report:		> 1_ATCACG_L001_R1_001_bismark_bt2_SE_report.txt <
Processing alignment report 1_ATCACG_L001_R1_001_bismark_bt2_SE_report.txt ...
Complete

Using the following deduplication report:	> 1_ATCACG_L001_R1_001_bismark_bt2.deduplication_report.txt <
Processing deduplication report 1_ATCACG_L001_R1_001_bismark_bt2.deduplication_report.txt ...
Complete

Using the following splitting report:		> 1_ATCACG_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt <
Processing splitting report 1_ATCACG_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt ...
Complete

Using the following M-bias report:		> 1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.M-bias.txt <
Processing M-bias report 1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.M-bias.txt ...
Co

In [32]:
%%bash
/Applications/bioinfo/Bismark_v0.19.0/bismark2summary

Found Bismark/Bowtie2 single-end files
No Bismark/Bowtie2 paired-end BAM files detected
No Bismark/Bowtie single-end BAM files detected
No Bismark/Bowtie paired-end BAM files detected

Generating Bismark summary report from 8 Bismark BAM file(s)...
>> Reading from Bismark report: 1_ATCACG_L001_R1_001_bismark_bt2_SE_report.txt
>> Reading from Bismark report: 2_CGATGT_L001_R1_001_bismark_bt2_SE_report.txt
>> Reading from Bismark report: 3_TTAGGC_L001_R1_001_bismark_bt2_SE_report.txt
>> Reading from Bismark report: 4_TGACCA_L001_R1_001_bismark_bt2_SE_report.txt
>> Reading from Bismark report: 5_ACAGTG_L001_R1_001_bismark_bt2_SE_report.txt
>> Reading from Bismark report: 6_GCCAAT_L001_R1_001_bismark_bt2_SE_report.txt
>> Reading from Bismark report: 7_CAGATC_L001_R1_001_bismark_bt2_SE_report.txt
>> Reading from Bismark report: 8_ACTTGA_L001_R1_001_bismark_bt2_SE_report.txt

Wrote Bismark project summary to >> bismark_summary_report.html <<



In [33]:
%%bash
rsync --progress -avz ../ /Volumes/web/seashell/bu-orange/


building file list ... 
206 files to consider
./
.DS_Store
        8196 100%    0.00kB/s    0:00:00 (xfer#1, to-check=204/206)
18-06-13/
18-06-13/.DS_Store
        6148 100%  600.39kB/s    0:00:00 (xfer#2, to-check=202/206)
18-06-13/1_ATCACG_L001_R1_001_bismark_bt2.bam
     1749181 100%   10.69MB/s    0:00:00 (xfer#3, to-check=201/206)
18-06-13/1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.M-bias.txt
        2756 100%   15.21kB/s    0:00:00 (xfer#4, to-check=200/206)
18-06-13/1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.bam
     1748959 100%    5.62MB/s    0:00:00 (xfer#5, to-check=199/206)
18-06-13/1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.bedGraph
       21209 100%   64.72kB/s    0:00:00 (xfer#6, to-check=198/206)
18-06-13/1_ATCACG_L001_R1_001_bismark_bt2.deduplicated.bismark.cov.gz
        5605 100%   16.39kB/s    0:00:00 (xfer#7, to-check=197/206)
18-06-13/1_ATCACG_L001_R1_001_bismark_bt2.deduplicated_splitting_report.txt
         785 100%    2.29kB/s    0:00:00 (xfer#8, to-ch